In [116]:
#  Read about Log. regression here 
#  https://en.wikipedia.org/wiki/Logistic_regression


In [1]:
import findspark
findspark.init()
import pyspark
from random import random
from pyspark import SparkContext

import sys
import re
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.util import MLUtils
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD,LogisticRegressionModel, LogisticRegressionWithLBFGS, SVMWithSGD

In [2]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '10')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]

4054 bkg318


In [3]:
sc = pyspark.SparkContext(appName="Logistic Regression", master='spark://polyp1:7077',conf = config )

In [4]:
dataFile = '/scratch/ISE495/lab6/data/enron.libsvm'
dictionary = '/scratch/ISE495/lab6/data/dictionary.dat'
IDFileMapping = '/scratch/ISE495/lab6/data/fileids.dat'

In [5]:
data = sc.textFile(dataFile)
data1 = sc.textFile(dictionary)
data2 = sc.textFile(IDFileMapping)

print data.take(1)
# print data1.count()
# print data2.take(1)

[u'14174 0 2354:1 4930:1 16751:2 24305:2 29014:1 49781:1 50345:1 68800:1 71782:1 89812:1 108348:1 114434:1 116870:1 136310:1 148421:1']


In [6]:
# read file "dictionary" and create a hashTable "dic" where KEY=ID and Value = word, e.g. dic[0]=fawn
dic={} 
with open(dictionary,'rU') as f:
    for l in f:
        d = l.strip().split(",")
        dic[int(d[0])]=d[1]
d = len(dic)  #TODO: size of dictionary
print "we have %d uniq words "%d

we have 157309 uniq words 


In [7]:
import numpy as np

import findspark
findspark.init()
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector

from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD, LogisticRegressionWithLBFGS, SVMWithSGD, NaiveBayes
from pyspark.mllib.tree import DecisionTree, GradientBoostedTrees, RandomForest


In [8]:
import string
import numpy as np

# read file "dataFile" and create a data  (fix this template code)
# dataFeatures=[]
# dataLabels=[]
# with open(dataFile,'rU') as f:
#     for line in f:
#         line= line.replace(':',' ')
#         data=line.split(' ')
#         label = int(data.pop(1))*2 - 1
#         #print label
#         data.pop(0)
#         ids=[int(data[x]) for x in range(0,len(data)) if x % 2==0]
#         #print ids
#         vals=[int(data[x]) for x in range(0,len(data)) if x % 2!=0]
#         #print vals
#         vals=vals/np.linalg.norm(vals)
#         #print vals
#         features = SparseVector(d,ids,vals)
#         #print features
#         dataFeatures+=[features]
#         dataLabels+=[label]
#     reutun LabeledPoint(dataLabels,dataFeatures)

In [9]:
#dataLabels

In [10]:
data.take(2)

[u'14174 0 2354:1 4930:1 16751:2 24305:2 29014:1 49781:1 50345:1 68800:1 71782:1 89812:1 108348:1 114434:1 116870:1 136310:1 148421:1',
 u'32720 1 128:1 137:1 203:1 975:1 2354:3 2675:1 5702:1 5845:1 6244:1 7515:1 11289:1 13682:1 13963:1 20991:1 21229:1 21779:1 21867:1 21872:1 23186:1 26852:1 27145:2 27704:1 28082:2 29959:1 30171:2 31087:1 31889:1 32720:1 34182:1 36102:1 36182:1 36565:1 36566:1 36591:2 39098:1 41188:1 46618:1 46969:1 47934:1 49551:1 49781:2 50575:2 51860:1 54655:2 55866:1 56324:3 56470:1 58658:1 59008:1 60121:1 60667:1 62775:1 62792:5 65227:1 68004:1 68245:1 68530:2 70199:1 71782:2 74994:1 75959:1 75972:1 80089:1 80241:1 80948:1 84072:1 84787:2 86528:1 86858:1 87285:1 97018:1 100864:1 101332:2 103095:1 103379:1 103506:1 106451:1 107293:1 112087:1 112509:1 112844:1 113425:1 113787:1 114029:2 114434:1 114848:1 116357:1 119133:1 122662:1 123601:1 124257:1 130124:1 130983:1 131255:1 132990:1 134321:1 134837:1 138265:1 144733:1 154296:1 157122:1']

In [21]:
def parseData(line):
    line= line.replace(':',' ')
    data=line.split(' ')
    label = int(data.pop(1))*2 - 1
    if label==-1:
        label=0
        #print label
    data.pop(0)
    ids=[int(data[x]) for x in range(0,len(data)) if x % 2==0]
        #print ids
    vals=[int(data[x]) for x in range(0,len(data)) if x % 2!=0]
        #print vals
    vals=vals/np.linalg.norm(vals)
        #print vals
    #dic{ids} = vals 
    features = SparseVector(d,ids,vals)
    return LabeledPoint(label,features)

parsedData = data.map(parseData)
print parsedData.take(1)


[LabeledPoint(0.0, (157309,[2354,4930,16751,24305,29014,49781,50345,68800,71782,89812,108348,114434,116870,136310,148421],[0.2182178902359924,0.2182178902359924,0.4364357804719848,0.4364357804719848,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924]))]


In [22]:

# this will split randomly the data 60% for training and 40% for testing
trainingData, testData = parsedData.randomSplit( [0.8, 0.2] , seed = 11L)
trainingData.cache()
testData.cache()
print trainingData.take(1)

[LabeledPoint(0.0, (157309,[2354,4930,16751,24305,29014,49781,50345,68800,71782,89812,108348,114434,116870,136310,148421],[0.2182178902359924,0.2182178902359924,0.4364357804719848,0.4364357804719848,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924,0.2182178902359924]))]


In [23]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [24]:
def score(model):
    predictions = model.predict(testData.map(lambda x: x.features))
    labels_and_preds = testData.map(lambda x: x.label).zip(predictions)
    accuracy = labels_and_preds.filter(lambda x: float(x[0]) == float(x[1])).count() / float(testData.count())
    return (accuracy*100)



In [25]:
#LogisticRegressionWithSGD
algo = LogisticRegressionWithSGD()
model = algo.train(trainingData)
score(model)





/opt/spark-2.2.0-bin-hadoop2.7/python/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


91.76628810520025

In [26]:
#LogisticRegressionWithLBFGS()
algo = LogisticRegressionWithLBFGS()
model = algo.train(trainingData)
score(model)

98.75971309025702

In [ ]:
#DecisionTree()
algo = DecisionTree()
model = algo.trainClassifier(trainingData, 2,{},impurity='entropy', maxDepth=5, maxBins=8)
score(model)

In [ ]:
# # Find more inforamtion about parameteres at:
# # https://spark.apache.org/docs/2.1.1/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS
# model = LogisticRegressionWithLBFGS.train(trainingData, iterations=15, regParam=0.0001, regType='l2', intercept=True)#, numClasses=10)


In [ ]:
# # Evaluating the model on training data
# labelsAndPreds = trainingData.map(lambda p: (p.label, model.predict(p.features)))
# trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(trainingData.count())
# print("Training Error = " + str(trainErr))

In [ ]:
# labelsAndPreds1 = testData.map(lambda p: (p.label, model.predict(p.features)))
# testErr = labelsAndPreds1.filter(lambda (v, p): v != p).count() / float(testData.count())
# print("Testing Error = " + str(testErr))

In [40]:
#model.predict([1.0,817.180406555498,12106.1347003149])

In [17]:
# now, create a copy of this jupyter notebook and run Log. Regression on the "SPAM" dataset

In [ ]:
# Hyper-parameter search:


In [ ]:
#The effect of normalization:
If dont normalize the 

In [ ]:
#Interpretation of the results:

In [ ]:
#Categorical Features in Decision Trees (DT):

In [150]:
sc.stop()